In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os.path
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipyleaflet import Map, Marker, AwesomeIcon
from IPython.display import clear_output

In [2]:
from _init import *

In [3]:
data_set = Dataset()
cleaning= Data_cleaning()
rambling = Data_rambling()
general_info= General_data()

In [ ]:
df=data_set.df_set(preprocess=True)
df.columns
general_info.df = df
rm = rest_models(df, "", False)

In [ ]:
@widgets.interact_manual(x=["apartment_room_number", "apartment_floor", "building_year", "price", "area_total"])
def plot(x="apartment_room_number"):
    general_info.box_plot(df, x)

In [ ]:
@widgets.interact_manual(x=["apartment_room_number", "apartment_floor", "building_year", "price", "area_total"])
def plot(x="apartment_room_number"):
    general_info.single_box_plot(df, x)

In [ ]:
@widgets.interact_manual(x=["apartment_room_number", "apartment_floor", "building_year", "price", "area_total"])
def plot(x="apartment_room_number"):
    general_info.hists(df, x)

In [ ]:
general_info.feature_importants(df)

In [ ]:
center = (52.0, 20.0)
estate = AwesomeIcon(name='home',marker_color='red',icon_color='white',spin=False)
m = Map(center=center, zoom=5)
marker = Marker(icon=estate, location=center, draggable=True)
m.add_layer(marker);
display(m)

In [ ]:
building_w = widgets.IntSlider(value=2000,min=1930,max=2021,step=1,description='Rok budowy:',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
display(building_w)

In [ ]:
area_total_w = widgets.BoundedFloatText(value=10.0, max=500.0, description='Powierzchnia:', disabled=False)
display(area_total_w)

In [ ]:
apartment_room_number_w = widgets.IntSlider(value=1,min=1,max=10,step=1, description='Liczba pokoi:',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
display(apartment_room_number_w)

In [ ]:
apartment_floor_w = widgets.IntSlider(value=0,min=0,max=25,step=1,description='Piętro:',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
display(apartment_floor_w)

In [ ]:
model_w = widgets.Button(description='Sprawdź cenę',disabled=False,button_style='success')
output = widgets.Output()

display(model_w, output)

def on_button_clicked(b):
    with output:
        try:
            apartment_floor = apartment_floor_w.value
            apartment_room_number = apartment_room_number_w.value
            area_total = area_total_w.value
            building_year = building_w.value 
            lat = marker.location[0] 
            long = marker.location[1]
            x_user = pd.DataFrame.from_dict({
                "fixed_latitude": lat,
                "fixed_longitude": long, 
                "building_year": building_year,
                "area_total": area_total, 
                "apartment_room_number": apartment_room_number, 
                "apartment_floor": apartment_floor
            })
            models = rm.load_model("regression", load_type="joblib", return_type="dict")
            for k, m in models.items():
                y_pred = m.predict(x_user)
                print(f"przewidziana cena: {y_pred} dla modelu {k}")
        except Exception as e:
            print(e)
model_w.on_click(on_button_clicked)

In [ ]:
model_classifier_w = widgets.Button(description='Pokaż rekomendacje',disabled=False,button_style='warning')
output_class = widgets.Output()

display(model_classifier_w, output_class)
def on_buttonclassifier_clicked(b):
    with output_class:
        try:
            t = TTT()
#             models = rm.load_ball()
#             print(models)
        except Exception as e:
            print(e)
model_classifier_w.on_click(on_buttonclassifier_clicked)